# Graph representation for the UL network
* import static data from files
* use networkx (and plotly) to represent data as nodes & edges

In [4]:
import csv

stops = {}
stopsOnTrip = {}
routeStations = {}
routeTrips = {}
tripRoute = {}
routeShortNames = {}

with open('../gtfs_static_data_ul/stops.csv') as file:
    reader = csv.DictReader(file)
    # Create dictionary key for each row in stops.csv with values on name, latitude and longitude
    for row in reader:
        stops[row['stop_id']] = {'stop_id': row['stop_id'], 'stop_name': row['stop_name'],
                                 'stop_lat': row['stop_lat'], 'stop_lon': row['stop_lon']}

with open('../gtfs_static_data_ul/stop_times.csv') as file:
    reader1 = csv.DictReader(file)
    # Loops through stop_times.csv and creates dictionary with stops per unique trip
    for row in reader1:
        stopsOnTrip.setdefault(row['trip_id'],[]).append(stops[row['stop_id']])
        
with open('../gtfs_static_data_ul/trips.csv') as file:
    reader3 = csv.DictReader(file)
    # Creates a dictionary with all unique trip_id:s per unique route_id
    for row in reader3:
        routeTrips.setdefault((row['direction_id'],row['route_id']),[]).append(row['trip_id'])
        
with open('../gtfs_static_data_ul/routes.csv') as file:
    reader4 = csv.DictReader(file)
    # Creates a dictionary with all unique route_id:s and corresponding route_short_name
    for row in reader4:
        routeShortNames.setdefault(row['route_id'],[]).append(row['route_short_name'])
        
with open('../gtfs_static_data_ul/trips.csv') as file:
    reader5 = csv.DictReader(file)
    # Creates a dictionary with all unique trip_id:s per unique route_id
    for row in reader5:
        tripRoute[row['trip_id']] = { 'route':row['route_id'],'direction':row['direction_id'] }
        
    # Creates dictionary with all station data per route
for route in routeTrips:
    routeStations[route] = { 'stops': stopsOnTrip[routeTrips[route][0]] }
    #routeStations.setdefault([route],[]).append(stopsOnTrip[routeTrips[route][0]])

## Creating of graph using networkx
* 5116 nodes
* 6229 edges
* 414 routes in total.

In [5]:
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import plot

edgeList = []
locDict = {}
graphObject = nx.DiGraph()
edge_x = []
edge_y = []
node_adjacencies = []
node_text = []

# Create list of all edges in the network
for routeID in routeStations.keys():
    for (i,stop) in enumerate(routeStations[routeID]['stops']):
        if i+1 < len(routeStations[routeID]['stops']):
            edgeList.append((routeStations[routeID]['stops'][i]['stop_id'], 
                     routeStations[routeID]['stops'][i+1]['stop_id']))

# Create edges (and indirectly nodes) for the graphObject
for edge in edgeList:
    graphObject.add_edge(edge[0],edge[1],weight=1)

# Establish dictionary with location data for stops in the network.
# One key per node and value as (lon, lat)
for node in graphObject.nodes():
    locDict[node]=(stops[node]['stop_lon'],stops[node]['stop_lat'])

# Loop through each edge in edgeList and add location data (x0,y0,x1,y1)
for edge in edgeList:
    x0, y0 = (stops[edge[0]]['stop_lon'],stops[edge[0]]['stop_lat'])
    x1, y1 = (stops[edge[1]]['stop_lon'],stops[edge[1]]['stop_lat'])
    
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

# Visualize edges using plotly graph_objects method Scatter()
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Loop through each node in graphObject and add location data (x,y)
node_x = []
node_y = []
for node in graphObject.nodes():
    x, y = (stops[node]['stop_lon'],stops[node]['stop_lat'])
    node_x.append(x)
    node_y.append(y)

# Visualize nodes using plotly graph_objects method Scatter()
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

# Count each nodes' adjencies in order to color code it
for node, adjacencies in enumerate(graphObject.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))

# Loop through each node and add its stop_name as text on hover
for i, node_id in enumerate(graphObject.nodes()):
    node_text.append(stops[node_id]['stop_name'] + ' ' + str(stops[node_id]['stop_id']))

# Connect the metadata variables to the node_trace go.Scatter() object 
node_trace.marker.color = node_adjacencies
node_trace.text = node_text

# Visualize everything in the figure
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Uppsala Public Transport Network',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
plot(fig, auto_open=True)

'temp-plot.html'

In [8]:
len(graphObject.nodes())

5145